## 配置文件

In [6]:
import sys, os
from os import path
sys.path.append('/root/bdrisk/risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

## 潜在风险模型
### 清洗模块

In [41]:
def clean_cr3():
    
    # 获取企业潜在风险相关数据指标
    POTENTIAL_RISK_DETAIL = Read_Oracle().read_oracle(sql= 
    """ 
    -- 单位全部参保人数是否正常
    select TYSHXYM ORG_CODE, DWMC ORG_NAME, cbny INDEX_DATE, '员工参保率' INDEX_NAME, round(DWCBRS/ZZZGPJRS,2) INDEX_VALUE
    from ods_zmxpq.ZWY_DWCBQK_XXB
    where TYSHXYM = '{}'
    and ZZZGPJRS != 0
    
    -- 员工平均参保金额是否正常
    union all
    select TYSHXYM ORG_CODE, DWMC ORG_NAME, cbny INDEX_DATE, '员工平均参保金额' INDEX_NAME, round(JNJE/DWCBRS,2) INDEX_VALUE
    from ods_zmxpq.ZWY_DWCBQK_XXB
    where TYSHXYM = '{}'
    
    -- 参保人口结构是否稳定
    union all
    select TYSHXYM ORG_CODE, DWMC ORG_NAME, cbny INDEX_DATE, '参保人数本市户籍占比' INDEX_NAME, round((BSCZHJJFRS+BSNCHJJFRS)/DWCBRS,2) INDEX_VALUE
    from ods_zmxpq.ZWY_DWCBQK_XXB
    where TYSHXYM = '{}'
    
    -- 水费
    union all
    select uni_sc_id ORG_CODE, org_name ORG_NAME, stat_date INDEX_DATE, '应收水费' INDEX_NAME, round(replace(water_rate,','),2) INDEX_VALUE
    from ods_zmxpq.WATER_RATE_USAGE t1
    left join
    (select uni_sc_id, corp_name
    from ods_zmxpq.corp_info where iscurrent = 1) t2
    on t1.org_name = t2.corp_name
    where uni_sc_id = '{}'
    
    -- 电费
    union all
    select uni_sc_id ORG_CODE, org_name ORG_NAME, stat_date INDEX_DATE, '应收电费' INDEX_NAME, round(replace(electric_charge,','),2)  INDEX_VALUE
    from ods_zmxpq.ELECTRIC_CHARGE_USAGE t3
    left join
    (select uni_sc_id, corp_name
    from ods_zmxpq.corp_info where iscurrent = 1) t4
    on t3.org_name = t4.corp_name
    where uni_sc_id = '{}'
    
    -- 煤气费
    union all
    select uni_sc_id ORG_CODE, org_name ORG_NAME, stat_date INDEX_DATE, '应收煤气费' INDEX_NAME, round(replace(gas_consumption,','),2)  INDEX_VALUE
    from ods_zmxpq.NATURAL_GAS_USAGE t5
    left join
    (select uni_sc_id, corp_name
    from ods_zmxpq.corp_info where iscurrent = 1) t6
    on t5.org_name = t6.corp_name
    where uni_sc_id = '{}'
    """.format('91310115688774070B', '91310115688774070B', '91310115688774070B', '91310115688774070B', '91310115688774070B', '91310115688774070B'), database = 'dbods')
    
    # 加入企业信息; ID; 模型运行时间
    POTENTIAL_RISK_DETAIL['ID'] = range(len(POTENTIAL_RISK_DETAIL))
    detail_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    POTENTIAL_RISK_DETAIL['CHECK_TIME'] = datetime.datetime.strptime(detail_now, "%Y-%m-%d %H:%M:%S")
    
    # 整理明细表
    POTENTIAL_RISK_DETAIL = POTENTIAL_RISK_DETAIL[['ID', 'ORG_CODE', 'INDEX_DATE', 'INDEX_NAME', 'INDEX_VALUE', 'CHECK_TIME']]
    
    # 读入数据库
    Write_Oracle().write_oracle('BD_RISK_DETAIL_CREDIT_CR3',POTENTIAL_RISK_DETAIL, org_code='91310115688774070B', alarm = None)
    
    
    # 用于前端画图
    CBRS = Read_Oracle().read_oracle(sql= 
        """ 
        select TYSHXYM ORG_CODE, cbny STA_DATE, DWCBRS CBRS, '位' CBRS_UNIT
        from ods_zmxpq.ZWY_DWCBQK_XXB
        where TYSHXYM = '91310115688774070B'
        """, database = 'dbods')
    ELECTRIC = Read_Oracle().read_oracle(sql= 
        """ 
        select uni_sc_id ORG_CODE, stat_date STA_DATE, round(replace(electric_charge,','),2) ELECTRIC, ELECTRIC_CHARGE_UNIT ELECTRIC_UNIT
        from ods_zmxpq.ELECTRIC_CHARGE_USAGE t3
        left join
        (select uni_sc_id, corp_name
        from ods_zmxpq.corp_info where iscurrent = 1) t4
        on t3.org_name = t4.corp_name
        where uni_sc_id = '91310115688774070B'
        """, database = 'dbods')
    I_E_VALUE = Read_Oracle().read_oracle(sql= 
        """ 
        with corp_goods as (select trade_code_scc, substr(DECL_DATE_KEY,1,6) as date_month, SUM_GOODS_GROSS_VALUE_RMB
        from DW_STA.FT_CUS_DWS_TRADE t1
        left join 
        DIM.DIM_TRADER t2 
        on t1.trade_code_key = t2.trade_key 
        where trade_code_scc = '91310115688774070B')
        select trade_code_scc ORG_CODE, date_month STA_DATE, round(sum(SUM_GOODS_GROSS_VALUE_RMB)/10000, 2) GOODS_VALUE, '万元' goods_unit
        from corp_goods
        where date_month >= '201801'
        group by trade_code_scc, date_month
        order by date_month
        """, database = 'dbdw')
    CREDIT_GRAPH = pd.merge(I_E_VALUE, CBRS, left_on=['ORG_CODE', 'STA_DATE'], right_on=['ORG_CODE', 'STA_DATE'], how='left')
    CREDIT_GRAPH = pd.merge(CREDIT_GRAPH, ELECTRIC, left_on=['ORG_CODE', 'STA_DATE'], right_on=['ORG_CODE', 'STA_DATE'], how='left')
    
    # 加入企业信息; ID; 模型运行时间
    CREDIT_GRAPH['ID'] = range(len(CREDIT_GRAPH))
    detail_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    CREDIT_GRAPH['CHECK_TIME'] = datetime.datetime.strptime(detail_now, "%Y-%m-%d %H:%M:%S")
    
    # 整理明细表
    CREDIT_GRAPH = CREDIT_GRAPH[['ID', 'ORG_CODE', 'STA_DATE', 'GOODS_VALUE', 'GOODS_UNIT', 'CBRS', 'CBRS_UNIT', 'ELECTRIC', 'ELECTRIC_UNIT', 'CHECK_TIME']]

    # 读入数据库
    Write_Oracle().write_oracle('BD_RISK_GRAPH_CREDIT_CR3',CREDIT_GRAPH, org_code='91310115688774070B', alarm = None)

In [42]:
clean_cr3()

2021-06-18 16:26:34.019 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.162s
2021-06-18 16:26:34.124 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.088s
2021-06-18 16:26:34.293 | INFO     | risk_models.config.write_config.write_func:write_oracle:181 - Processing... Writing 115 rows into database
2021-06-18 16:26:34.368 | INFO     | risk_models.config.write_config.write_func:write_oracle:188 - Insert data into BD_RISK_DETAIL_CREDIT_CR3 successfully! Total write time spent 0.337s
2021-06-18 16:26:34.483 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.113s
2021-06-18 16:26:34.598 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.112s
2021-06-18 16:26:37.733 | INFO     | risk_models.config.read_confi

In [86]:
detail

,ID,ORG_CODE,INDEX_DATE,INDEX_NAME,INDEX_VALUE,CHECK_TIME
0,0,91310115688774070B,201801,员工参保率,1.12,2021-06-16 15:41:02
1,1,91310115688774070B,201802,员工参保率,1.12,2021-06-16 15:41:02
2,2,91310115688774070B,201803,员工参保率,1.12,2021-06-16 15:41:02
3,3,91310115688774070B,201804,员工参保率,1.12,2021-06-16 15:41:02
4,4,91310115688774070B,201805,员工参保率,1.12,2021-06-16 15:41:02
...,...,...,...,...,...,...
110,110,91310115688774070B,202011,应收电费,52693.00,2021-06-16 15:41:02
111,111,91310115688774070B,202012,应收电费,60802.00,2021-06-16 15:41:02
112,112,91310115688774070B,202101,应收电费,56834.00,2021-06-16 15:41:02
113,113,91310115688774070B,202102,应收电费,55008.00,2021-06-16 15:41:02


In [44]:
graph

,ID,ORG_CODE,STA_DATE,GOODS_VALUE,GOODS_UNIT,CBRS,CBRS_UNIT,ELECTRIC,ELECTRIC_UNIT,CHECK_TIME
0,0,91310115688774070B,201801,96044.97,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
1,1,91310115688774070B,201802,70000.24,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
2,2,91310115688774070B,201803,74351.99,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
3,3,91310115688774070B,201804,30081.42,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
4,4,91310115688774070B,201805,11270.53,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
5,5,91310115688774070B,201806,6183.14,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
6,6,91310115688774070B,201807,4123.15,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
7,7,91310115688774070B,201808,2313.40,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
8,8,91310115688774070B,201809,538.13,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37
9,9,91310115688774070B,201810,587.41,万元,57.0,位,NaN,NaN,2021-06-18 16:26:37


### 计算模块

In [38]:
def model_cr3():
    
    # 读取明细表
    POTENTIAL_RISK_RESULT = Read_Oracle().read_oracle(sql= """ select * from BD_RISK_DETAIL_CREDIT_CR3 where org_code = '{}' 
                                                            and iscurrent = 1 """.format('91310115688774070B'), database = 'dbods')
    
    # 阈值计算函数
    def Qfunc(df):
        Q1 = np.percentile(df['INDEX_VALUE'], 25)
        Q3 = np.percentile(df['INDEX_VALUE'], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR
        return (Q3 + outlier_step), (Q1 - outlier_step)
    
    # 计算上下阈值
    cutoff = POTENTIAL_RISK_RESULT.groupby(['ORG_CODE', 'INDEX_NAME']).apply(Qfunc)
    cutoff = pd.DataFrame(cutoff).reset_index()
    cutoff['CUTOFF_HIGH'] = cutoff[0].map(lambda x: x[0])
    cutoff['CUTOFF_LOW'] = cutoff[0].map(lambda x: x[1])
    cutoff.drop(0, axis=1, inplace=True)
    POTENTIAL_RISK_RESULT = pd.merge(POTENTIAL_RISK_RESULT, cutoff, on=['ORG_CODE','INDEX_NAME'], how='left')
    
    # 打标签（'员工参保率'不采用IQR阈值，使用参数固定阈值）
    POTENTIAL_RISK_RESULT['RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '员工参保率','INDEX_VALUE'].map(lambda x: '员工参保率不足' if x <= 0.90 else '员工参保率正常')
    
    # 其余5类标签(采用IQR阈值)
    # 平均参保金额判断
    if POTENTIAL_RISK_RESULT[POTENTIAL_RISK_RESULT.INDEX_NAME == '员工平均参保金额'].empty is True:
        print('找不到该企业员工平均参保金额数据')
    else:
        cutoff_low_1 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='员工平均参保金额','CUTOFF_LOW'])[0]
        cutoff_high_1 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='员工平均参保金额','CUTOFF_HIGH'])[0]
        POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '员工平均参保金额','RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '员工平均参保金额','INDEX_VALUE'].map(lambda x: '人均参保金较低' if x <= cutoff_low_1 else ('人均参保金较高' if x >= cutoff_high_1 else '人均参保金额稳定'))
    
    # 参保人数本市户籍占比判断
    if POTENTIAL_RISK_RESULT[POTENTIAL_RISK_RESULT.INDEX_NAME == '参保人数本市户籍占比'].empty is True:
        print('找不到该企业参保人数本市户籍占比数据')
    else:
        cutoff_low_2 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='参保人数本市户籍占比','CUTOFF_LOW'])[0]
        cutoff_high_2 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='参保人数本市户籍占比','CUTOFF_HIGH'])[0]
        POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '参保人数本市户籍占比','RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '参保人数本市户籍占比','INDEX_VALUE'].map(lambda x: '本市户籍人数异常下降' if x <= cutoff_low_2 else ('本市户籍人数异常上升' if x >= cutoff_high_2 else '本市户籍人数稳定'))

    # 应收水费判断
    if POTENTIAL_RISK_RESULT[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收水费'].empty is True:
        print('找不到该企业应收水费数据')
    else:
        cutoff_low_3 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收水费','CUTOFF_LOW'])[0]
        cutoff_high_3 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收水费','CUTOFF_HIGH'])[0]
        POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收水费','RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收水费','INDEX_VALUE'].map(lambda x: '水资源消耗异常下降' if x <= cutoff_low_3 else ('水资源消耗异常上升' if x >= cutoff_high_3 else '水资源消耗水平正常'))
    
    # 应收电费判断
    if POTENTIAL_RISK_RESULT[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收电费'].empty is True:
        print('找不到该企业应收电费数据')
    else:
        cutoff_low_4 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收电费','CUTOFF_LOW'])[0]
        cutoff_high_4 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收电费','CUTOFF_HIGH'])[0]
        POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收电费','RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收电费','INDEX_VALUE'].map(lambda x: '电量消耗异常下降' if x <= cutoff_low_4 else ('电量消耗异常上升' if x >= cutoff_high_4 else '电量消耗水平正常'))
    
    # 应收煤气费判断
    if POTENTIAL_RISK_RESULT[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收煤气费'].empty is True:
        print('找不到该企业应收煤气费数据')
    else:
        cutoff_low_5 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收煤气费','CUTOFF_LOW'])[0]
        cutoff_high_5 = np.unique(cutoff.loc[cutoff.INDEX_NAME =='应收煤气费','CUTOFF_HIGH'])[0]
        POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收煤气费','RISK_LABEL'] = POTENTIAL_RISK_RESULT.loc[POTENTIAL_RISK_RESULT.INDEX_NAME == '应收煤气费','INDEX_VALUE'].map(lambda x: '煤气消耗异常下降' if x <= cutoff_low_5 else ('煤气消耗异常上升' if x >= cutoff_high_5 else '煤气消耗水平正常'))
    
    
    # 算分
    ratio = float(-50 / len(POTENTIAL_RISK_RESULT))
    good_sample = ['员工参保率正常', '人均参保金额稳定', '本市户籍人数稳定', '水资源消耗水平正常', '电量消耗水平正常', '煤气消耗水平正常']
    # 计算分数
    POTENTIAL_RISK_RESULT['SCORE'] = POTENTIAL_RISK_RESULT['RISK_LABEL'].map(lambda x: 0.00 if x in good_sample else ratio)
    
    # 更新ID; 模型运行时间
    POTENTIAL_RISK_RESULT['ID'] = range(len(POTENTIAL_RISK_RESULT))
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    POTENTIAL_RISK_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
    
    # 整理结果表
    POTENTIAL_RISK_RESULT = POTENTIAL_RISK_RESULT[['ID','ORG_CODE','INDEX_DATE','INDEX_NAME','RISK_LABEL','SCORE','CHECK_TIME']]
    
    # 读入数据库
    Write_Oracle().write_oracle('BD_RISK_RESULT_CREDIT_CR3',POTENTIAL_RISK_RESULT, org_code='91310115688774070B', alarm = None)
    
    # 整理预警明细数据，并写入数据库
    RISK_ALARM = POTENTIAL_RISK_RESULT[(POTENTIAL_RISK_RESULT.RISK_LABEL != '员工参保率正常')&(POTENTIAL_RISK_RESULT.RISK_LABEL != '人均参保金额稳定')&(POTENTIAL_RISK_RESULT.RISK_LABEL != '本市户籍人数稳定')&(POTENTIAL_RISK_RESULT.RISK_LABEL != '水资源消耗水平正常')&(POTENTIAL_RISK_RESULT.RISK_LABEL != '电量消耗水平正常')&(POTENTIAL_RISK_RESULT.RISK_LABEL != '煤气消耗水平正常')].groupby(['RISK_LABEL'], as_index=False)['ID'].count()
    RISK_ALARM = RISK_ALARM.rename(columns={'ID':'ALARM_NUMBER'})
    RISK_ALARM['ALARM_REASON'] = '发现' + RISK_ALARM['ALARM_NUMBER'].astype('str') + '起' + RISK_ALARM['RISK_LABEL'] + '事件'
    RISK_ALARM['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
    RISK_ALARM['ORG_CODE'] = '91310115688774070B'
    RISK_ALARM['MODEL_CODE'] = 'CREDIT'
    RISK_ALARM['CHILD_MODEL_CODE'] = 'CR3'
    RISK_ALARM['ID'] = range(len(RISK_ALARM))
    RISK_ALARM = RISK_ALARM[['ID','ORG_CODE','MODEL_CODE','CHILD_MODEL_CODE','ALARM_REASON','ALARM_NUMBER','CHECK_TIME']]
    
    if RISK_ALARM.empty:
        print('没有异常情况')
    else:
        Write_Oracle().write_oracle('BD_RISK_ALARM_ITEM',RISK_ALARM, org_code = '91310115688774070B', alarm = ['CREDIT','CR3'])
        

In [39]:
model_cr3()

2021-06-18 16:26:21.710 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 4.728s


找不到该企业应收煤气费数据


In [40]:
result

,ID,ORG_CODE,INDEX_DATE,INDEX_NAME,RISK_LABEL,SCORE,CHECK_TIME
0,0,91310115688774070B,201801,员工参保率,员工参保率正常,0.0,2021-06-18 16:26:21
1,1,91310115688774070B,201802,员工参保率,员工参保率正常,0.0,2021-06-18 16:26:21
2,2,91310115688774070B,201803,员工参保率,员工参保率正常,0.0,2021-06-18 16:26:21
3,3,91310115688774070B,201804,员工参保率,员工参保率正常,0.0,2021-06-18 16:26:21
4,4,91310115688774070B,201805,员工参保率,员工参保率正常,0.0,2021-06-18 16:26:21
...,...,...,...,...,...,...,...
110,110,91310115688774070B,202011,应收电费,电量消耗水平正常,0.0,2021-06-18 16:26:21
111,111,91310115688774070B,202012,应收电费,电量消耗水平正常,0.0,2021-06-18 16:26:21
112,112,91310115688774070B,202101,应收电费,电量消耗水平正常,0.0,2021-06-18 16:26:21
113,113,91310115688774070B,202102,应收电费,电量消耗水平正常,0.0,2021-06-18 16:26:21
